## Comprehensive data exploration with Python

『人生で最も難しいことは、自分自身を知ることだ』

この引用はミレトスのタレスに由来します。タレスはギリシャ／フェニキアの哲学者・数学者・天文学者であり、西洋文明において科学的思考を初めて実践した人物として知られています（出典：Wikipedia – Thales）。

データサイエンスにおいて「データを知ること」が人生で最も難しいことだとは言いませんが、時間がかかるのは事実です。そのため、この初期ステップを見落とし、早々に次のステップへ進んでしまうことがよくあります。

ですので、私は「水に飛び込む前に泳ぎ方を学ぶ」ことを試みました。Hair ら（2013年）の『データを調べる』という章を参考にして、包括的ではあるが完全ではない分析を試みました。これは厳密な研究報告というわけではありませんが、コミュニティにとって役立つことを願って、私がどのようにデータ分析の原則をこの問題に適用したかを共有することにしました。

章の名前は少々風変わりかもしれませんが、このカーネル（ノートブック）で行っていることは以下のようなものです：

- 問題を理解する。
　各変数を確認し、それらの意味やこの問題における重要性について哲学的な観点から分析します。

- 単変量分析。
　従属変数（'SalePrice'）のみに注目して、もう少し深く理解しようとします。

- 多変量分析。
　従属変数と独立変数の関係性を探り、それらがどのように関係しているかを理解しようとします。

- 基本的なクレンジング。
　データセットの欠損値、外れ値、カテゴリ変数を処理してクリーンアップします。

- 前提条件の検証。
　私たちのデータが、多くの多変量解析技法で要求される前提条件を満たしているかを確認します。

In [1]:
#invite people for the Kaggle party
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df_train = pd.read_csv('./data/train.csv')
df_train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

### 1. それで……何がわかるのでしょうか？


データを理解するために、各変数を見て、その意味やこの問題との関連性を考えてみましょう。これは時間のかかる作業ですが、データセットの「味わい」をつかむことができます。

分析に一定の規律をもたせるために、以下の列をもつExcelスプレッドシートを作成することができます：

Variable（変数） – 変数の名前。

Type（型） – 変数の型の識別。ここには2つの値が入ります：「数値型（numerical）」または「カテゴリ型（categorical）」。
「数値型」とは、値が数値である変数のことで、「カテゴリ型」とは、値がカテゴリ（分類）である変数のことです。

Segment（セグメント） – 変数の分類（セグメント）の識別。次の3つのセグメントに分類できます：
建物（building）・空間（space）・場所（location）。

「建物」とは、建物の物理的特性に関連する変数（例：OverallQual）。

「空間」とは、住宅の空間的特性を示す変数（例：TotalBsmtSF）。

「場所」とは、住宅がどこにあるかに関する情報を与える変数（例：Neighborhood）。

Expectation（予想） – その変数が「SalePrice（販売価格）」に与える影響についての予想。
「高（High）」「中（Medium）」「低（Low）」のカテゴリ尺度を使うことができます。

Conclusion（結論） – データをざっと見た後で、その変数の重要性についての結論。
「Expectation」と同様に、「高・中・低」のカテゴリ尺度で評価できます。

Comments（コメント） – その他、思いついた一般的なコメント。
